<a href="https://colab.research.google.com/github/GaboRamalho/Big-Data/blob/main/SDG_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Java

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Download and install Apache Spark:

In [ ]:
!wget -q !wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

Set environment variables to specify where Spark is installed:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

Include Spark using findpark.

In [ ]:
import findspark
findspark.init()

Start a local session and import data:

In [ ]:
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
#load data from computer
data_spark = sc.read.csv('/content/unsdg_2002_2021.csv', header = True)

In [ ]:
#verify the type of data in the dataset
type(data_spark)

pyspark.sql.dataframe.DataFrame

In [ ]:
#verify the dataset

data_spark.show(30)

+-------+----------+-------------+--------------------+--------------------+---------------------------------------+-----------------------------------------------+----------------------+-----------------------+---------------------------------------------------------------+-------------------+------------------------------------------------------+-------------------------------------------------------+----------------------------------------------------------+--------------------------------------+---------------------------------------------------+-----------------------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------+
|dt_year|   dt_date|      country|              region|level_of_development|greenhousegas_emissione_mtco2equivalent|total_government_revenue_proportion_of_gdp_perc|fdi_inflows_millionusd|annual_growth_rate_perc|proportion_of_population_with_primary_reliance_on_cl

In [ ]:
#verify the schema()
data_spark.printSchema()

root
 |-- dt_year: string (nullable = true)
 |-- dt_date: string (nullable = true)
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- level_of_development: string (nullable = true)
 |-- greenhousegas_emissione_mtco2equivalent: string (nullable = true)
 |-- total_government_revenue_proportion_of_gdp_perc: string (nullable = true)
 |-- fdi_inflows_millionusd: string (nullable = true)
 |-- annual_growth_rate_perc: string (nullable = true)
 |-- proportion_of_population_with_primary_reliance_on_clean_fuels_a: string (nullable = true)
 |-- mortality_rate_perc: string (nullable = true)
 |-- renewable_energy_share_on_the_total_energy_consumption: string (nullable = true)
 |-- co2emissions_from_fuel_combustion_in_millions_of_tonnes: string (nullable = true)
 |-- fossilfuel_subsidies_consumption_and_production_billionusd: string (nullable = true)
 |-- randd_expenditure_proportionofgdp_perc: string (nullable = true)
 |-- national_action_plans_as_priority_national_po

In [ ]:
#return how many lines it have
data_spark.count()

5176

In [ ]:
#select some columns
data_spark.select("dt_year", "country", "region", "level_of_development", "number_of_companies_publishing_sustainability_reports").show(30)

+-------+-------------+--------------------+--------------------+-----------------------------------------------------+
|dt_year|      country|              region|level_of_development|number_of_companies_publishing_sustainability_reports|
+-------+-------------+--------------------+--------------------+-----------------------------------------------------+
|   2002|  Afghanistan|Central and South...|     Least Developed|                                                 null|
|   2003|  Afghanistan|Central and South...|     Least Developed|                                                 null|
|   2004|  Afghanistan|Central and South...|     Least Developed|                                                 null|
|   2005|  Afghanistan|Central and South...|     Least Developed|                                                 null|
|   2006|  Afghanistan|Central and South...|     Least Developed|                                                 null|
|   2007|  Afghanistan|Central and South

In [ ]:
#main statistics
data_spark.describe().show()

+-------+------------------+----------+-------------+--------------------+--------------------+---------------------------------------+-----------------------------------------------+----------------------+-----------------------+---------------------------------------------------------------+-------------------+------------------------------------------------------+-------------------------------------------------------+----------------------------------------------------------+--------------------------------------+---------------------------------------------------+-----------------------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------+
|summary|           dt_year|   dt_date|      country|              region|level_of_development|greenhousegas_emissione_mtco2equivalent|total_government_revenue_proportion_of_gdp_perc|fdi_inflows_millionusd|annual_growth_rate_perc|proportion_of_

In [ ]:
#sql import to use group by
from pyspark.sql import functions as F

In [ ]:
# Group the data by "region" and "level_of_development"
grouped_data = data_spark.groupBy("region", "level_of_development")

# Apply an aggregation function to the grouped data (e.g., count)
result = grouped_data.count()

# Show the result
result.show(30)


+--------------------+--------------------+-----+
|              region|level_of_development|count|
+--------------------+--------------------+-----+
|Central and South...|Economies in Tran...|   98|
|Eastern and South...|     Least Developed|   60|
|               Other|       Not Specified|  800|
|Eastern and South...| Developed Economies|   20|
|Central and South...|     Least Developed|   80|
|Central and South...|Developing Economies|   80|
|Europe and North ...|Economies in Tran...|  160|
|             Oceania|       Not Specified|  100|
|               Other|     Least Developed|   80|
|Eastern and South...|       Not Specified|   60|
|North Africa and ...|       Not Specified|   60|
|Latin America and...|Developing Economies|  460|
|Central and South...|       Not Specified|   20|
|             Oceania| Developed Economies|   40|
|               Other|Developing Economies|   20|
|Europe and North ...| Developed Economies|  700|
|Eastern and South...|Developing Economies|  200|


In [ ]:
from pyspark.sql.functions import col


country = "Brazil"

# Filter data for the specific country and years 2002 and 2021
filtered_data = data_spark.filter((col("country") == country) & (col("dt_year").between(2002, 2021)))

# analyze or aggregate the filtered data as needed
filtered_data.show()  # Display the filtered data, or perform further calculations


+-------+----------+-------+--------------------+--------------------+---------------------------------------+-----------------------------------------------+----------------------+-----------------------+---------------------------------------------------------------+-------------------+------------------------------------------------------+-------------------------------------------------------+----------------------------------------------------------+--------------------------------------+---------------------------------------------------+-----------------------------------------------------+-------------------------------------+----------------------------------------------------+----------------------------------+
|dt_year|   dt_date|country|              region|level_of_development|greenhousegas_emissione_mtco2equivalent|total_government_revenue_proportion_of_gdp_perc|fdi_inflows_millionusd|annual_growth_rate_perc|proportion_of_population_with_primary_reliance_on_clean_fuels_a|

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Replace 'Brazil' with the name of the country you want to analyze
country = "Brazil"

# Filter data for the specific country and years 2002 to 2021
filtered_data = data_spark.filter((col("country") == country) & (col("dt_year").between(2002, 2021)))

# Group the filtered data by the SDG
grouped_data = filtered_data.groupBy("greenhousegas_emissione_mtco2equivalent")

# Calculate statistics for each SDG, for example, average values of relevant columns
result = grouped_data.avg("greenhousegas_emissione_mtco2equivalent")

# Convert the result to a Pandas DataFrame for visualization
result_pandas = result.toPandas()

# plot the data using Matplotlib
# Assuming you want to compare 'column1' and 'column2' for each SDG
plt.figure(figsize=(12, 6))
for column in ["avg(greenhousegas_emissione_mtco2equivalent)"]:
    plt.plot(result_pandas['sdg'], result_pandas[column], label=column)
plt.xlabel('SDG')
plt.ylabel('Average Value')
plt.title(f'Evolution of SDGs 1 for {country} (2002-2021)')
plt.legend()
plt.grid(True)
plt.show()


AnalysisException: ignored